# 전체 코드 실행 후 테스트 코드를 실행해주세요.

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
import pandas as pd
import numpy as np
import rhinoMorph
# Loading en_vocab_list
with open("/content/drive/MyDrive/Ai_test/1000common_list_en.txt", "r") as file:
    lines = file.read().split(',')
# Loading Rihno
rn = rhinoMorph.startRhino()
# Setting Device
device = torch.device("cpu")
# Loading Kobert Vocab
_, vocab = get_pytorch_kobert_model()

filepath:  /usr/local/lib/python3.6/dist-packages
classpath:  /usr/local/lib/python3.6/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
using cached model
using cached model


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768, num_classes=9, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# Loading Model
model = torch.load('/content/drive/MyDrive/Ai_test/model_KoBert_RhinoDataRefine.pt', map_location=device)

In [ ]:
def del_unimportant_n(sentence, lines, rn):
  sentence = sentence.replace('\r','')
  sentence = sentence.replace('\n','')
  sentence = sentence.replace('\([^)]*\)', '')
  sentence = str.upper(sentence)
  sentence = sentence.replace("[^가-힣A-Z0-9=%\. ]","")
  del_dic = {}
  n_list = rhinoMorph.onlyMorph_list(rn, sentence, pos=['SL'])
  for n in n_list:
    if len(n)>1:
      if n not in lines:
        del_dic[n] = ''
  if len(del_dic)!=0:
    sentence = pd.Series(sentence)
    new_sentence = sentence.replace(del_dic, regex=True)
  else:
    new_sentence = pd.Series(sentence)
  new_sentence = new_sentence.replace({' +':' '}, regex=True)
  return new_sentence[0]

def BERT_change(sentence) :
    sentence = del_unimportant_n(sentence, lines, rn)
    tokenizer= nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)
    transform = nlp.data.BERTSentenceTransform(tokenizer, max_seq_length = 128, pad = True, pair = False)
    sentence_val = transform([sentence])
    inputs = torch.tensor(sentence_val[0], dtype=torch.long)
    inputs = inputs.unsqueeze(0)
    lens = torch.tensor(sentence_val[1])
    lens = lens.unsqueeze(0)
    masks = torch.tensor(sentence_val[2])
    return inputs, lens, masks

def test_sentences(sentences):
    model.eval()
    inputs, lens, masks = BERT_change(sentences)
    b_input_ids = inputs.to(device)
    b_lens = lens.to(device)
    b_masks = masks.to(device)
    with torch.no_grad():     
        outputs = model(b_input_ids, valid_length = b_lens, segment_ids = b_masks)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    tagging = np.argmax(logits)
    dicts_data = {0 : '가설 설정', 1 : '기술 정의', 2 : '대상 데이터', 3 : '데이터처리',
                  4 : '문제 정의', 5 : '성능/효과', 6 : '이론/모형', 7 : '제안 방법', 8 : '후속연구'}
    tagging = dicts_data[tagging]
    return tagging

# 테스트코드

In [ ]:
while True:
  x = input("태깅할 문장을 하나씩 입력하세요 (종료:0) : ")
  if x=='0':
    break
  print('문장 태깅 결과 :',test_sentences(x))
  print('-'*170)

태깅할 문장을 하나씩 입력하세요 (종료:0) : 본 연구에서는 상수도 직결형 스프링클러 시스템의 성능을 평가하기 위하여 실물 주택을 대상으로 화재실험을 수행하였다.
using cached model
문장 태깅 결과 : 제안방법
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
태깅할 문장을 하나씩 입력하세요 (종료:0) : 철킬레이트 촉매를 이용한 2가나 3가 철을 사용하여 황화수소를 제거하기 위해 철염의 농도를 달리하여 제거효율을 측정하여 보았다.
using cached model
문장 태깅 결과 : 제안방법
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
태깅할 문장을 하나씩 입력하세요 (종료:0) : 특히 질량 흐름식을 사용하여 불확도를 계산하고 농도에 대한 합성불확도를 구하였다.
using cached model
문장 태깅 결과 : 제안방법
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
태깅할 문장을 하나씩 입력하세요 (종료:0) : 먼저 모듈 부품에서의 VOCs 방출량을 확인하기 위해 ISO 12219-4 (Small chamber method)를 사용 하였다.
using cached mode